# Getting Started
This notebook will walk you through on how to create a portfolio using csv data, run it through a backtester using a simple a strategy, and view the resulting statistics

First we must **import** all of our dependancies. Pat-Analytics has five distinct sub-modules:
- Core : contains the Market and Portfolio objects

- Strategy : contains the strategy classes

- Backtester : contains the backtester logic

- Metrics : Computes basic metrics about the portfolio/market

- Models : Sophisticated models module  

This tutorial will walk you through a basic usage of the first four

In [1]:
from pathlib import Path
import pandas as pd

from pat_analytics import Market, Portfolio
from pat_analytics.strategy import BuyNHold, StratConfig
from pat_analytics.backtesters import Backtester
from pat_analytics.metrics import RiskReport, PerformanceReport

## Workflow  
To start, the user will initiate
1. a Market object containing all market data, 

2. a portfolio object which contains what, how much, and when a position is held, 

3. a strategy which picks a target weight at each timestep for the portfolio, 

4. the backtester which executes the trades given by the strategy,

5. finally the metrics and models which assist the strategy in stock picking.  

  
Let's do that now!

### Load in Data
Begin with the Market Class, this is the authoritative container of _all_ raw observable market data. This modules inputs are :
- price_data          					: MultiIndex Dataframe [(ticker, type) x time] -> price (dollar amount)
  
- meta_data (**OPTIONAL**)           	: Dataframe [ticker x data_type] -> value (sector, country, currency etc)

- fundemental_data (**OPTIONAL**)    	: MultiIndex DataFrame [(ticker, type) x time] -> value () 

- dividend_data(**OPTIONAL**)	       	: DataFrame [ticker x time] -> dividend yield

- macro_data(**OPTIONAL**)				: DataFrame [t-bill type x time] -> yield

- fx_data(**OPTIONAL**)             	: DataFrame [currency x time] -> price (1 Currency price in USD)

You have multiple ways of loading in the data: 
- from_dict() 			: pass in a dictionary of dataframes containing price data 

- from_sql()			: not implemented yet

- from_alphavantage() 	: not implemented yet  

This tutorial will use the from dict method, using csv   
  
Do note, we create a fake position called 'CASH' filled with 1s, this is to keep track of the users CASH in their portfolio and make the rest of the dataframe operations easier on us :p. A better solution will be thought up of later (hopefully).

In [2]:
parent_dir = Path.cwd().parent #relative import
data_dir = parent_dir / "sample-data"

tickers = ["AAPL", "SPY", "LULU"]
price_data = {}
for s in tickers:
    df = pd.read_csv(data_dir / f"{s}.csv")
    df['datetime'] = pd.to_datetime(df['epoch'], unit='s')
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    price_data[s] = df

market = Market.from_dict(price_data)


### Portfolio  
The Portfolio object houses the most basic of portfolio information, and provides some utility methods for portfolio manipulation. The main component of the Portfolio are 
- weight 	: DataFrame[ticker x time] -> weight (number from 0 - 1 showing the weight of the positition in the portfolio at a time)

- quantity	: DataFrame[ticker x time] -> quantity (either a real or an integer number representing the amount of actual shares of a position at a time)
  
We wish to find these quantities at each timestep t, thru the backtester. Initiate the portfolio by inputing:  
- tickers             			: list[str] (list of ticker names, or any unique identifiers, a subset of the market tickers)

- init_weight (**OPTIONAL*** )	: pd.Series [ticker] -> weight  | str (the weight of each position in the portfolio at the start)

- init_quantity(**OPTIONAL*** )	: pd.Series [ticker] -> quantity of stock (the quantity of each position in the portfolio at the start)

- init_market_value(**OPTIONAL**): float (the starting market value of the portfolio in USD)

(*) Users must provide either weight XOR quantity, exactly one.  

The Portfolio weight and quantity will always contain a column for housing the amount of cash (under the name 'CASH'), if the user did not include it, it will be done for them. The user may pass one of weight or quantity, as with the weight and market information we cna infer the quantity, and with the quantity and market information we can infer the weight. This is done for flexibility. Moreover, the initial weight can be initialized with predefined options :  
  
**INITIAL WEIGHT OPTIONS**  
- A __pandas Series__
- the string __'uniform'__ which will give a weight of $\dfrac{1}{n}$ for each position exising in the Market, where $n = $ # positions in the market + 1 (for cash)

For now this portfolio will hold a uniform weight

In [3]:
port = Portfolio(market, init_weight='uniform')


### Strategy  

In [7]:
config = StratConfig()
strat = BuyNHold(config=config)



### Backtester  


In [8]:
bt = Backtester(port, market, strat)

bt.run()

ValueError: setting an array element with a sequence.